In [ ]:
## https://huggingface.co/hfl/llama-3-chinese-8b
## https://huggingface.co/hfl/llama-3-chinese-8b/tree/main

In [1]:
import random, os, tqdm, time, json
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../")

random.seed(618)
np.random.seed(907)

new_base_path = os.path.join(
    "/Users/minkexiu/Downloads/",
    "/".join(
        os.getcwd().split("/")[-1*(len(sys.path[-1].split("/")) - 1):]
    ),
)
print("storage dir:", new_base_path)
print("code dir:", os.getcwd())

## 创建文件夹。
if not os.path.exists(new_base_path):
    os.makedirs(
        new_base_path
    )
if not os.path.exists(os.path.join(new_base_path, "preprocessedData")):
    os.makedirs(
        os.path.join(new_base_path, "preprocessedData")
    )
if not os.path.exists(os.path.join(new_base_path, "originalData")):
    os.makedirs(
        os.path.join(new_base_path, "originalData")
    )
if not os.path.exists(os.path.join(new_base_path, "trained_models")):
    os.makedirs(
        os.path.join(new_base_path, "trained_models")
    )

def create_originalData_path(filename_or_path):
    return os.path.join(new_base_path, "originalData", filename_or_path)
def create_preprocessedData_path(filename_or_path):
    return os.path.join(new_base_path, "preprocessedData", filename_or_path)
def create_trained_models_path(filename_or_path):
    return os.path.join(new_base_path, "trained_models", filename_or_path)

def millisec2datetime(timestamp):
    time_local = time.localtime(timestamp/1000)
    return time.strftime("%Y-%m-%d %H:%M:%S", time_local)
    
def run_finish():
    # 假设你的字体文件是 'myfont.ttf' 并且位于当前目录下  
    font = FontProperties(fname="/Users/minkexiu/Documents/GitHub/ML_Tryout/SimHei.ttf", size=24)  
    # 创建一个空白的图形  
    fig, ax = plt.subplots()  
    ax.imshow(
        plt.imread("/Users/minkexiu/Downloads/wallhaven-dgxpyg.jpg")
    )
    # 在图形中添加文字  
    ax.text(
        ax.get_xlim()[1] * 0.5, 
        ax.get_ylim()[0] * 0.5, 
        f"程序于这个点跑完：\n{millisec2datetime(time.time()*1000)}", fontproperties=font, ha="center", va="center", color="red"
    )  
    # 设置图形的布局  
    # ax.set_xlim(0, 1)  
    # ax.set_ylim(0, 1)  
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.patch.set_color("blue")
    # 显示图形  
    plt.show()
        
tqdm.tqdm.pandas() ## 引入这个，就可以在apply的时候用progress_apply了。

import IPython
def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True) 
    
def simply_show_data(df1):
    print(df1.shape)
    display(df1.head())
    
def wait_flag(saved_flag_path, time_interval_sec=10):
    print("waiting for", saved_flag_path)
    time_count = 0
    while True:
        if os.path.exists(saved_flag_path):
            break
        time.sleep(time_interval_sec)
        time_count+=time_interval_sec
        print(time_count, end=" ")
    print("finish!!")

storage dir: /Users/minkexiu/Downloads/GitHub/ML_runCodeFromBook/HuggingFace自然语言处理
code dir: /Users/minkexiu/Documents/GitHub/ML_runCodeFromBook/HuggingFace自然语言处理


In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="hfl/llama-3-chinese-8b", cache_dir = create_originalData_path(""))

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/78/08/78088ba0cd2ae5322048a5de69cff85a44e3c0ab637f6708fd9f48eaa6624335/502323771ce6f0aaf6022e39e4ffdeb8bed40046636a09ae3d457e7719916b78?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1717333025&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMzMzAyNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc4LzA4Lzc4MDg4YmEwY2QyYWU1MzIyMDQ4YTVkZTY5Y2ZmODVhNDRlM2MwYWI2MzdmNjcwOGZkOWY0OGVhYTY2MjQzMzUvNTAyMzIzNzcxY2U2ZjBhYWY2MDIyZTM5ZTRmZmRlYjhiZWQ0MDA0NjYzNmEwOWFlM2Q0NTdlNzcxOTkxNmI3OD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=LJdlCcjErqutTfvpMttMKF-HQfGlqtA6RHfFPhwdVPTpAGbxdOyhZmhYM19MPo3FC-ZUJUZIDi%7EJ3MW-6ZQq3wWETaApkIDUavFrg%7EAVT6LSzOS%7EKaxT0%7EnCSxO-op7W2neS00ZuQkq1zj3waxRFd%7E9hLdDk0%7EObX6Cf%7EnI-8ixFd9RXzq29AQWZknfn%7E

model-00002-of-00004.safetensors:   3%|3         | 157M/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/78/08/78088ba0cd2ae5322048a5de69cff85a44e3c0ab637f6708fd9f48eaa6624335/502323771ce6f0aaf6022e39e4ffdeb8bed40046636a09ae3d457e7719916b78?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1717333657&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMzMzY1N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc4LzA4Lzc4MDg4YmEwY2QyYWU1MzIyMDQ4YTVkZTY5Y2ZmODVhNDRlM2MwYWI2MzdmNjcwOGZkOWY0OGVhYTY2MjQzMzUvNTAyMzIzNzcxY2U2ZjBhYWY2MDIyZTM5ZTRmZmRlYjhiZWQ0MDA0NjYzNmEwOWFlM2Q0NTdlNzcxOTkxNmI3OD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=N-Nkw%7EN-p3N9XJjQpwkTE7aReNQETYOMTOqMEQmg2Dxz9WMIT8hiL9Gts%7Eqo-3kzq3D2CHA0rqlk%7E9NVbgRupkhIArin59MVjTNmXIFs1KptK13eSGWlPzpB5CqADaLaj03xNzOLFlqbbB7nV2Rf%7E46mtFin1wtb9yDV4LgXsMNy5vDUCNGpURTzKC3P0WGLdc4Iw

...

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/78/08/78088ba0cd2ae5322048a5de69cff85a44e3c0ab637f6708fd9f48eaa6624335/502323771ce6f0aaf6022e39e4ffdeb8bed40046636a09ae3d457e7719916b78?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1717333657&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMzMzY1N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc4LzA4Lzc4MDg4YmEwY2QyYWU1MzIyMDQ4YTVkZTY5Y2ZmODVhNDRlM2MwYWI2MzdmNjcwOGZkOWY0OGVhYTY2MjQzMzUvNTAyMzIzNzcxY2U2ZjBhYWY2MDIyZTM5ZTRmZmRlYjhiZWQ0MDA0NjYzNmEwOWFlM2Q0NTdlNzcxOTkxNmI3OD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=N-Nkw%7EN-p3N9XJjQpwkTE7aReNQETYOMTOqMEQmg2Dxz9WMIT8hiL9Gts%7Eqo-3kzq3D2CHA0rqlk%7E9NVbgRupkhIArin59MVjTNmXIFs1KptK13eSGWlPzpB5CqADaLaj03xNzOLFlqbbB7nV2Rf%7E46mtFin1wtb9yDV4LgXsMNy5vDUCNGpURTzKC3P0WGLdc4Iw

model-00002-of-00004.safetensors:  15%|#5        | 765M/5.00G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/78/08/78088ba0cd2ae5322048a5de69cff85a44e3c0ab637f6708fd9f48eaa6624335/502323771ce6f0aaf6022e39e4ffdeb8bed40046636a09ae3d457e7719916b78?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1717333657&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMzMzY1N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc4LzA4Lzc4MDg4YmEwY2QyYWU1MzIyMDQ4YTVkZTY5Y2ZmODVhNDRlM2MwYWI2MzdmNjcwOGZkOWY0OGVhYTY2MjQzMzUvNTAyMzIzNzcxY2U2ZjBhYWY2MDIyZTM5ZTRmZmRlYjhiZWQ0MDA0NjYzNmEwOWFlM2Q0NTdlNzcxOTkxNmI3OD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=N-Nkw%7EN-p3N9XJjQpwkTE7aReNQETYOMTOqMEQmg2Dxz9WMIT8hiL9Gts%7Eqo-3kzq3D2CHA0rqlk%7E9NVbgRupkhIArin59MVjTNmXIFs1KptK13eSGWlPzpB5CqADaLaj03xNzOLFlqbbB7nV2Rf%7E46mtFin1wtb9yDV4LgXsMNy5vDUCNGpURTzKC3P0WGLdc4Iw

model-00002-of-00004.safetensors:  18%|#7        | 891M/5.00G [00:00<?, ?B/s]

ValueError: Could not load model hfl/llama-3-chinese-8b with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ml12/lib/python3.12/site-packages/urllib3/response.py", line 737, in _error_catcher
    yield
  File "/opt/anaconda3/envs/ml12/lib/python3.12/site-packages/urllib3/response.py", line 862, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/ml12/lib/python3.12/site-packages/urllib3/response.py", line 845, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/ml12/lib/python3.12/http/client.py", line 472, in read
    s = self.fp